In [9]:
from statsmodels.imputation import mice
from statsmodels.formula.api import ols
import statsmodels.api as sm
import numpy as np
import pandas as pd

# Visualizing Missing Data


In [3]:
available_data = pd.read_csv(r'/home/beastman/Projects/HomeSchool/data-analysis/BehavioralDataAnalysis/Chapter 6 - Handling missing data/chap6-available_data.csv')

In [4]:
from itertools import combinations

# building a 4-axis function equivalent to the R function
# top: columns sorted by number of missing values
# bottom: total missing value counts per column
# left: count of non-null rows across combinations of variables
# right: count of combinations of columns with null values (unnecessary)
def md_pattern(df: pd.DataFrame):
    null_counts = df.isna().sum().sort_values()
    null_list = null_counts[null_counts.values > 0].keys()

    viz = pd.DataFrame(columns = list(null_counts.keys())+ ["null count", "depth","col_nulls"])

    r1b = pd.Series(data = dict.fromkeys(list(null_counts.keys()), " ")).to_frame().T
    r1b["null count"] = len(df.dropna())
    r1b["depth"] = len(null_list)
    r1b["col_nulls"] = 0

    viz = pd.concat([viz, r1b])

    for a in range(1,len(null_list) + 1):
        comb = [list(x) for x in combinations(null_list, a)]

        for c in comb:
            bool_null = dict.fromkeys(list(null_counts.keys()), " ")
            depth = list(null_list).index(c[0])
            search_list = c
            include =  list()
            exclude = list()

            for i, x in enumerate(search_list):
                
                index_list = list(df[df[x].isnull()].index)
                if i == 0:
                    include = index_list
                else:
                    for item in include[::-1]:
                        if item not in index_list:
                            include.remove(item)                

            for y in df.columns.drop(search_list):
                exclude += list(df[df[y].isnull()].index)

            null_count = len(df[(df.index.isin(include)) & ~(df.index.isin(exclude))])

            if null_count > 0:
                for col in c:
                    bool_null[col] = "X"
                
                col_nulls = sum("X" == x for x in list(bool_null.values()))
                bool_null["col_nulls"] = col_nulls
                bool_null["null count"] = null_count
                bool_null["depth"] = depth
                

                row = pd.Series(data = bool_null).to_frame().T

                viz = pd.concat([viz, row], ignore_index=True)
            else: 
                break

    vizdf = (viz.set_index("depth")).sort_values(by=["depth"], ascending=[False]).reset_index(drop=True)
    return vizdf, null_counts

viz, counts = md_pattern(available_data)

viz

,age,open,gender,bkg_amt,state,extra,neuro,null count,col_nulls
0,,,,,,,,368,0
1,,,,,,,X,358,1
2,,,,,,X,,249,1
3,,,,,,X,X,228,2
4,,,,,X,,,163,1
5,,,,,X,X,,125,2
6,,,,,X,,X,214,2
7,,,,,X,X,X,120,3
8,,,,X,,,,33,1
9,,,,X,X,,,24,2


In [5]:
counts

age           0
open          0
gender        0
bkg_amt     175
state       711
extra       793
neuro      1000
dtype: int64

In [12]:
min_data_df = available_data.copy()
min_data_df.neuro = np.where(min_data_df.neuro.isna(), min_data_df.neuro.min(), min_data_df.neuro)

max_data_df = available_data.copy()
max_data_df.neuro = np.where(max_data_df.neuro.isna(), max_data_df.neuro.max(), max_data_df.neuro)


for data in [available_data, min_data_df, max_data_df]:
    print(ols("bkg_amt~neuro", data=data).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                bkg_amt   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     7.492
Date:                Thu, 26 Oct 2023   Prob (F-statistic):            0.00632
Time:                        15:27:55   Log-Likelihood:                -5187.4
No. Observations:                 905   AIC:                         1.038e+04
Df Residuals:                     903   BIC:                         1.039e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    258.0886     14.306     18.041      0.0